Airline Delay Prediction : SVR 

In [48]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [49]:
data = pd.read_csv("airline_2m 2.csv",encoding='Iso-8859-1', low_memory=False,nrows=500)
pd.set_option('display.max_columns', None)

In [50]:
data.head()

,Year,Quarter,Month,DayofMonth,DayOfWeek,FlightDate,Reporting_Airline,DOT_ID_Reporting_Airline,IATA_CODE_Reporting_Airline,Tail_Number,Flight_Number_Reporting_Airline,OriginAirportID,OriginAirportSeqID,OriginCityMarketID,Origin,OriginCityName,OriginState,OriginStateFips,OriginStateName,OriginWac,DestAirportID,DestAirportSeqID,DestCityMarketID,Dest,DestCityName,DestState,DestStateFips,DestStateName,DestWac,CRSDepTime,DepTime,DepDelay,DepDelayMinutes,DepDel15,DepartureDelayGroups,DepTimeBlk,TaxiOut,WheelsOff,WheelsOn,TaxiIn,CRSArrTime,ArrTime,ArrDelay,ArrDelayMinutes,ArrDel15,ArrivalDelayGroups,ArrTimeBlk,Cancelled,CancellationCode,Diverted,CRSElapsedTime,ActualElapsedTime,AirTime,Flights,Distance,DistanceGroup,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay,FirstDepTime,TotalAddGTime,LongestAddGTime,DivAirportLandings,DivReachedDest,DivActualElapsedTime,DivArrDelay,DivDistance,Div1Airport,Div1AirportID,Div1AirportSeqID,Div1WheelsOn,Div1TotalGTime,Div1LongestGTime,Div1WheelsOff,Div1TailNum,Div2Airport,Div2AirportID,Div2AirportSeqID,Div2WheelsOn,Div2TotalGTime,Div2LongestGTime,Div2WheelsOff,Div2TailNum,Div3Airport,Div3AirportID,Div3AirportSeqID,Div3WheelsOn,Div3TotalGTime,Div3LongestGTime,Div3WheelsOff,Div3TailNum,Div4Airport,Div4AirportID,Div4AirportSeqID,Div4WheelsOn,Div4TotalGTime,Div4LongestGTime,Div4WheelsOff,Div4TailNum,Div5Airport,Div5AirportID,Div5AirportSeqID,Div5WheelsOn,Div5TotalGTime,Div5LongestGTime,Div5WheelsOff,Div5TailNum
0,1998,1,1,2,5,1998-01-02,NW,19386,NW,N297US,675,13487,1348701,31650,MSP,"Minneapolis, MN",MN,27,Minnesota,63,14869,1486902,34614,SLC,"Salt Lake City, UT",UT,49,Utah,87,1640,1659.0,19.0,19.0,1.0,1.0,1600-1659,24.0,1723.0,1856.0,3.0,1836,1859.0,23.0,23.0,1.0,1.0,1800-1859,0.0,NaN,0.0,176.0,180.0,153.0,1.0,991.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2009,2,5,28,4,2009-05-28,FL,20437,FL,N946AT,671,13342,1334202,33342,MKE,"Milwaukee, WI",WI,55,Wisconsin,45,13204,1320401,31454,MCO,"Orlando, FL",FL,12,Florida,33,1204,1202.0,-2.0,0.0,0.0,-1.0,1200-1259,10.0,1212.0,1533.0,8.0,1541,1541.0,0.0,0.0,0.0,0.0,1500-1559,0.0,NaN,0.0,157.0,159.0,141.0,1.0,1066.0,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2013,2,6,29,6,2013-06-29,MQ,20398,MQ,N665MQ,3297,11921,1192102,31921,GJT,"Grand Junction, CO",CO,8,Colorado,82,11298,1129803,30194,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,1630,1644.0,14.0,14.0,0.0,0.0,1600-1659,9.0,1653.0,1936.0,6.0,1945,1942.0,-3.0,0.0,0.0,-1.0,1900-1959,0.0,NaN,0.0,135.0,118.0,103.0,1.0,773.0,4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010,3,8,31,2,2010-08-31,DL,19790,DL,N6705Y,1806,12892,1289201,32575,LAX,"Los Angeles, CA",CA,6,California,91,11433,1143301,31295,DTW,"Detroit, MI",MI,26,Michigan,43,1305,1305.0,0.0,0.0,0.0,0.0,1300-1359,23.0,1328.0,2008.0,7.0,2035,2015.0,-20.0,0.0,0.0,-2.0,2000-2059,0.0,NaN,0.0,270.0,250.0,220.0,1.0,1979.0,8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2006,1,1,15,7,2006-01-15,US,20355,US,N504AU,465,11618,1161801,31703,EWR,"Newark, NJ",NJ,34,New Jersey,21,11057,1105702,31057,CLT,"Charlotte, NC",NC,37,North Carolina,36,1820,1911.0,51.0,51.0,1.0,3.0,1800-1859,19.0,1930.0,2050.0,8.0,2026,2058.0,32.0,32.0,1.0,2.0,2000-2059,0.0,NaN,0.0,126.0,107.0,80.0,1.0,529.0,3,0.0,0.0,0.0,0.0,32.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [51]:
data.shape

(500, 109)

In [52]:
df2 = data.loc[:, data.isnull().mean() < .8]

In [53]:
df11= df2.dropna()

In [54]:
df11= df11.drop(['FlightDate'], axis=1)
df11= df11.drop(['ArrDelay'], axis=1)

In [55]:
from sklearn.preprocessing import LabelEncoder

lb_make = LabelEncoder()
df11["Reporting_Airline_code"] = lb_make.fit_transform(df11["Reporting_Airline"])

In [56]:
df11=df11.drop(['Reporting_Airline'], axis=1)
df11=df11.drop(['IATA_CODE_Reporting_Airline'], axis=1)
df11=df11.drop(['OriginCityName'], axis=1)
df11=df11.drop(['OriginStateName'], axis=1)
df11=df11.drop(['DestCityName'], axis=1)
df11=df11.drop(['DestStateName'], axis=1)
df11=df11.drop(['DepTimeBlk'], axis=1)
df11=df11.drop(['ArrTimeBlk'], axis=1)

In [57]:
lb_make = LabelEncoder()
df11["Origin"] = lb_make.fit_transform(df11["Origin"])
df11["OriginState"] = lb_make.fit_transform(df11["OriginState"])
df11["Dest"] = lb_make.fit_transform(df11["Dest"])
df11["DestState"] = lb_make.fit_transform(df11["DestState"])

In [58]:
df11["Tail_Number"] = lb_make.fit_transform(df11["Tail_Number"])

In [59]:
df11.shape

(178, 47)

In [60]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(df11.select_dtypes(include=numerics).columns)
df12 = df11[numerical_vars]

In [61]:
from sklearn.model_selection import train_test_split

In [62]:
X_train, X_test, y_train, y_test = train_test_split(
    df12.drop(labels=['ArrDelayMinutes'], axis=1),
    df12['ArrDelayMinutes'],
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((124, 46), (54, 46))

In [63]:
X = df12[['DepDelay', 'DepDelayMinutes', 'DepartureDelayGroups']]
y = df12['ArrDelayMinutes']

In [64]:
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler

In [65]:
cv = KFold(n_splits=4, random_state=42, shuffle=False)
for train_index, test_index in cv.split(X):
    print("Train Index: ", train_index, "\n")
    print("Test Index: ", test_index)

Train Index:  [ 45  46  47  48  49  50  51  52  53  54  55  56  57  58  59  60  61  62
  63  64  65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80
  81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 113 114 115 116
 117 118 119 120 121 122 123 124 125 126 127 128 129 130 131 132 133 134
 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149 150 151 152
 153 154 155 156 157 158 159 160 161 162 163 164 165 166 167 168 169 170
 171 172 173 174 175 176 177] 

Test Index:  [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44]
Train Index:  [  0   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17
  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35
  36  37  38  39  40  41  42  43  44  90  91  92  93  94  95  96  97  98
  99 100 101 102 103 104 105 106 107 108 109 110 111 112 11

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:297: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  FutureWarning


In [66]:
# Initialise a new scaling object for normalising input data
sc=StandardScaler() 

# Set up the scaler just on the training set
sc.fit(X_train)
sc.fit(X_test)


# Apply the scaler to the training and test sets
X_train_std=sc.transform(X_train)
X_test_std=sc.transform(X_test)


In [67]:
from sklearn.svm import SVR

In [68]:
best_svr = SVR(kernel='linear')

In [69]:
best_svr.fit(X_train_std, y_train)

SVR(kernel='linear')

In [70]:
preds = best_svr.predict(X_test_std)

In [71]:
preds

array([ -1.08222312,  -3.74381038,   0.846959  ,   0.78431194,
        -6.53915589,   7.67719126,  -7.14442579,   7.1752305 ,
         7.58004512,  -4.27005298,   0.5721769 ,   8.89163431,
         0.42026356,  17.12860716,  25.72674073,  -3.24126825,
         8.12119888,  28.4107649 ,  24.96870268,  -1.19986708,
        -3.98966091,   9.37448975,  20.65038778,   6.78743911,
        34.73202989,  -0.42106536,  11.14866606,  62.80817165,
       114.19939634,  10.51235187,  -2.18796266,  -3.94380344,
         3.97477341,  -7.5198814 ,  -2.63909336,  -7.64815644,
         0.27091313,  -2.1305905 ,  12.27292271,  -2.67703609,
         9.26165235,   5.29883002,  -5.37550154,  -4.4517137 ,
         8.9899946 , 123.06130403,  -3.54400607,  -0.79449116,
         5.09714545,  -7.58930714,   1.02392209,  23.96168688,
        28.01041898,   2.93385336])

In [72]:
from sklearn import metrics
from sklearn.metrics import mean_absolute_error

In [73]:
mean_absolute_error(y_test,preds)

5.476103879183946